In [2]:
# Here you can play with specific components of the system.

In [ ]:
# Method for searching anime via AniList API
from anilist_query_searcher import search_anime

search_results = search_anime(
    # search_term="Attack on titan",
    # genres=["Sports", "Ecchi"],  # Using single genre parameter
    tags=["Volleyball"],
    per_page=5
)

# # Print all results first
print("All results:")
for anime in search_results:
    print(f"- {anime['title']['english'] or anime['title']['romaji']}")
    print(f"  Genres: {', '.join(anime['genres'])}")
    print(f"  Tags: {', '.join([tag['name'] for tag in anime['tags']])}")
    print("-" * 60)

All results:
- HAIKYU!!
  Genres: Comedy, Drama, Sports
  Tags: Volleyball, School Club, Male Protagonist, Primarily Male Cast, Ensemble Cast, Shounen, Primarily Teen Cast, School, Coming of Age, Fitness, Cute Boys Doing Cute Things, Kuudere
------------------------------------------------------------
- HAIKYU!! 2nd Season
  Genres: Comedy, Drama, Sports
  Tags: Volleyball, School Club, Ensemble Cast, Primarily Teen Cast, Primarily Male Cast, Shounen, Male Protagonist, Coming of Age, School, Tomboy
------------------------------------------------------------
- HAIKYU!! 3rd Season
  Genres: Comedy, Drama, Sports
  Tags: Volleyball, School Club, Shounen, Male Protagonist, Primarily Male Cast, Primarily Teen Cast, Ensemble Cast, School, Coming of Age
------------------------------------------------------------
- HAIKYU!! TO THE TOP
  Genres: Comedy, Drama, Sports
  Tags: Volleyball, Male Protagonist, Shounen, School Club, Primarily Male Cast, Ensemble Cast, School, Coming of Age, Primaril

In [6]:
# Stage 1 - Request Parser
from crewai import Agent, Task, LLM
import os
from request_parser import AnimeSearchParams, OFFICIAL_GENRES

# Initialize LLM (using GPT-4o from OpenAI)
llm = LLM(
    model="gpt-4.1",
    api_key=os.getenv('OPENAI_API_KEY'),
    max_tokens=2000,
    temperature=0
)

mapper = Agent(
    name="AniListRequestMapper",
    role="Filter extractor",
    goal="Return only the filters in the user's request, as minimal JSON.",
    backstory="An anime librarian who knows the difference between genres and tags.",
    allow_delegation=False,
    llm=llm
)

map_request = Task(
    description=(
        "USER_REQUEST:\n"
        "{user_request}\n\n"
        "Produce **one JSON object** that validates against AnimeSearchParams.\n"
        "Rules:\n"
        f"• Only items in this list may appear in `genres`: {OFFICIAL_GENRES}.\n"
        "• Any other descriptive phrase (moods, sub‑genres like 'School Life', "
        "adjectives like 'Wholesome') must go into `tags`.\n"
        "• Title‑case every word (e.g. 'school‑life' → 'School Life').\n"
        "Don't necessarily include genres unless they are specified in the request.\n"
        "• Omit every key whose value would be null.\n\n"
        "Example:\n"
        "USER_REQUEST:  Recommend a dark fantasy from 2020.\n"
        "OUTPUT: {\"genres\": [\"Fantasy\"], \"tags\": [\"Dark\"], \"year\": 2020}"
    ),
    expected_output="A JSON dict with only the mentioned filters, no nulls.",
    output_json=AnimeSearchParams,
    agent=mapper,
)

In [ ]:
# Testing for stage 1 - Request Parser
from crewai import Crew, Process

prompt = "I want an isekai anime with some comedy"

crew = Crew(
    agents=[mapper],
    tasks=[map_request],
    process=Process.sequential,
)
crew.kickoff(inputs={"user_request": prompt})

params = {k: v for k, v in map_request.output.json_dict.items() if v is not None}
print(params)



{'genres': ['Comedy'], 'tags': ['Isekai']}


In [9]:
# Stage 2 - Anime searcher and recommender
from recommender import SearchAnimeTool
from crewai import Agent, Crew, Task

anime_tool = SearchAnimeTool()

anime_researcher = Agent(
    role="Anime Researcher",
    goal="Find animes that match a user query.",
    backstory="You are a seasoned anime critic.",
    tools=[anime_tool],
)

recommendation_task = Task(
    description="Using AniList, compile a list of five anime that match the query.",
    expected_output="Five recommendations with one‑sentence justifications.",
    agent=anime_researcher,
)

In [10]:
# Arming the whole crew: Stage 1 + Stage 2
crew = Crew(
    agents=[mapper, anime_researcher],
    tasks=[map_request, recommendation_task],
    process=Process.sequential,
)

In [ ]:
# Testing the whole system

In [11]:
result = crew.kickoff(inputs={"user_request": "I want an anime like Akira"})
print(result)

Here are five anime recommendations based on your interest in "Akira":

1. **Akira (1988)**  
   An iconic cyberpunk film that explores themes of power and societal collapse, showcasing stunning animation and a haunting soundtrack that defines the genre.  
   ![Akira](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx47-4CR68arv452h.jpg)

2. **Battle Angel Alita (GUNNM) (1993)**  
   This OVA adaptation introduces a cyborg protagonist in a dystopian future, combining action and emotional depth with a rich backstory, reminiscent of "Akira."  
   ![Battle Angel Alita](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx1016-3tyU3Y3KtcGA.png)

3. **AKIRA (Shin Anime) (Upcoming)**  
   A new adaptation of the original story, this series is anticipated to explore the same haunting themes of dystopia and human evolution present in the classic film.  
   ![AKIRA (Shin Anime)](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx126016-9PaIQHxFyyho.png)

4.

In [13]:
result = crew.kickoff(inputs={"user_request": "I want an anime like ghost in the shell"})
print(result)

1. **PSYCHO-PASS** - This anime explores a dystopian society governed by a system that judges the mental state of citizens, fitting perfectly with the themes of surveillance and societal control found in "Ghost In The Shell."  
   ![PSYCHO-PASS](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx13601-i42VFuHpqEOJ.jpg)

2. **Vivy: Fluorite Eye’s Song** - A unique blend of action and musical elements, it delves into a future influenced by AI, highlighting existential questions akin to those in "Ghost In The Shell."  
   ![Vivy: Fluorite Eye’s Song](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx128546-UIwyhuhjxmL0.jpg)

3. **Ghost in the Shell: Stand Alone Complex** - A continuation of the franchise, this series expands on the original narrative, further exploring themes of identity and consciousness through a cyberpunk lens.  
   ![Ghost in the Shell: Stand Alone Complex](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx467-mBTtIoR13qs2.jpg)

In [12]:
result = crew.kickoff(inputs={"user_request": "I want an tennis anime with some comedy"})
print(result)

1. **The Prince of Tennis (Tennis no Ouji-sama)** - This iconic sports and comedy anime blends intense tennis matches with engaging character development, making it an essential watch for sports enthusiasts. ![The Prince of Tennis](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx22-yEguU9EmxkjK.png)

2. **Teekyuu** - A fast-paced and hilarious take on tennis, this short episode series offers surreal humor and comedic shortcuts while showcasing the fun of school tennis club life. ![Teekyuu](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx15125-2ngZyS4KrMqA.png)

3. **Teekyuu 2** - The sequel to Teekyuu continues the absurd comedy and character interactions, maintaining the quirky charm that made the first season a success while pushing the humor to new heights. ![Teekyuu 2](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx18121-k8UaPii7p9sl.jpg)

4. **Prince of Tennis II (Shin Tennis no Ouji-sama)** - This follow-up series revitalizes the o